In [2]:
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import os
from PIL import Image
import pickle
from random import shuffle
import albumentations as A
import torch
torch.__version__
from torch import nn
import torch.nn.functional as F
from torchvision.datasets import STL10
import torchvision.transforms.functional as tvf
from torchvision import transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, SequentialSampler
import random
from PIL import Image
from torch.multiprocessing import cpu_count
from torch.optim import RMSprop
from torch.optim.lr_scheduler import ReduceLROnPlateau
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from argparse import Namespace

import pandas as pd
df = pd.read_csv("/workstation/home/bijoy/data_from_b170007ec/Programs/Bhanu/SCLEARNING/Output_csv/Model_binary_v3.csv", usecols=range(1,5))
predicted_noise = df[df["Predicted_label"]==1]
cervix = df[(df["Predicted_label"] == 0)]
cervix = cervix[cervix["Data_type"] == "Train"]

test_ext = pd.read_csv("/workstation/home/bijoy/data_from_b170007ec/Programs/Bhanu/SCLEARNING/TESTING_ON_5DATASETS/testing_data.csv")

imagenetdata = test_ext[test_ext["imagenet"] == 1]
blooddata = test_ext[test_ext["blood"] == 1]
eyedata = test_ext[test_ext["eye"] == 1]
skindata = test_ext[test_ext["skin"] == 1]
surgerydata = test_ext[test_ext["surgery"] == 1]

img_dirs = []
labels_temp = []
for a in list(predicted_noise["Path"]):
    img_dirs.append(a)
    labels_temp.append(0)
for b in list(cervix["Path"]):
    img_dirs.append(b)
    labels_temp.append(1)
for j in list(imagenetdata["Path"]):
    img_dirs.append(j)
    labels_temp.append(2)
for k in list(blooddata["Path"]):
    img_dirs.append(k)
    labels_temp.append(3)
for l in list(eyedata["Path"]):
    img_dirs.append(l)
    labels_temp.append(4)
for m in list(skindata["Path"]):
    img_dirs.append(m)
    labels_temp.append(5)
for n in list(surgerydata["Path"]):
    img_dirs.append(n)
    labels_temp.append(6)

data = img_dirs
labels = labels_temp
# data = data_temp
# labels = labels_temp
print("Length of data :",len(data))
print("Length of labels :",len(labels))



class SupConLoss(nn.Module):
    """Supervised Contrastive Learning: https://arxiv.org/pdf/2004.11362.pdf.
    It also supports the unsupervised contrastive loss in SimCLR"""
    def __init__(self, temperature=0.07, contrast_mode='all',
                 base_temperature=0.07):
        super(SupConLoss, self).__init__()
        self.temperature = temperature
        self.contrast_mode = contrast_mode
        self.base_temperature = base_temperature

    def forward(self, features, labels=None, mask=None):
        """Compute loss for model. If both `labels` and `mask` are None,
        it degenerates to SimCLR unsupervised loss:
        https://arxiv.org/pdf/2002.05709.pdf
        Args:
            features: hidden vector of shape [bsz, n_views, ...].
            labels: ground truth of shape [bsz].
            mask: contrastive mask of shape [bsz, bsz], mask_{i,j}=1 if sample j
                has the same class as sample i. Can be asymmetric.
        Returns:
            A loss scalar.
        """
        device = (torch.device('cuda')
                  if features.is_cuda
                  else torch.device('cpu'))

        if len(features.shape) < 3:
            raise ValueError('`features` needs to be [bsz, n_views, ...],'
                             'at least 3 dimensions are required')
        if len(features.shape) > 3:
            features = features.view(features.shape[0], features.shape[1], -1)

        batch_size = features.shape[0]
        if labels is not None and mask is not None:
            raise ValueError('Cannot define both `labels` and `mask`')
        elif labels is None and mask is None:
            mask = torch.eye(batch_size, dtype=torch.float32).to(device)
        elif labels is not None:
            labels = labels.contiguous().view(-1, 1)
            if labels.shape[0] != batch_size:
                raise ValueError('Num of labels does not match num of features')
            mask = torch.eq(labels, labels.T).float().to(device)
        else:
            mask = mask.float().to(device)

        contrast_count = features.shape[1]
        contrast_feature = torch.cat(torch.unbind(features, dim=1), dim=0)
        if self.contrast_mode == 'one':
            anchor_feature = features[:, 0]
            anchor_count = 1
        elif self.contrast_mode == 'all':
            anchor_feature = contrast_feature
            anchor_count = contrast_count
        else:
            raise ValueError('Unknown mode: {}'.format(self.contrast_mode))

        # compute logits
        anchor_dot_contrast = torch.div(
            torch.matmul(anchor_feature, contrast_feature.T),
            self.temperature)
        # for numerical stability
        logits_max, _ = torch.max(anchor_dot_contrast, dim=1, keepdim=True)
        logits = anchor_dot_contrast - logits_max.detach()

        # tile mask
        mask = mask.repeat(anchor_count, contrast_count)
        # mask-out self-contrast cases
        logits_mask = torch.scatter(
            torch.ones_like(mask),
            1,
            torch.arange(batch_size * anchor_count).view(-1, 1).to(device),
            0
        )
        mask = mask * logits_mask
        
        # compute log_prob
        exp_logits = torch.exp(logits) * logits_mask
        log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))
       
        # compute mean of log-likelihood over positive
        mean_log_prob_pos = (mask * log_prob).sum(1) / mask.sum(1)
     
        # loss
        loss = - (self.temperature / self.base_temperature) * mean_log_prob_pos
        loss = loss.view(anchor_count, batch_size).mean()

        return loss
    
def random_rotate(image):
    if random.random() > 0.5:
        return tvf.rotate(image, angle=random.choice((0, 90, 180, 270)))
    return image

class ResizedRotation():
    def __init__(self, angle, output_size=(512,512)):
        self.angle = angle
        self.output_size = output_size
        
    def angle_to_rad(self, ang): return np.pi * ang / 180.0
        
    def __call__(self, image):
        w, h = image.size
        new_h = int(np.abs(w * np.sin(self.angle_to_rad(90 - self.angle))) + np.abs(h * np.sin(self.angle_to_rad(self.angle))))
        new_w = int(np.abs(h * np.sin(self.angle_to_rad(90 - self.angle))) + np.abs(w * np.sin(self.angle_to_rad(self.angle))))
        img = tvf.resize(image, (new_w, new_h))
        img = tvf.rotate(img, self.angle)
        img = tvf.center_crop(img, self.output_size)
        return img
    
class WrapWithRandomParams():
    def __init__(self, constructor, ranges):
        self.constructor = constructor
        self.ranges = ranges
    
    def __call__(self, image):
        randoms = [float(np.random.uniform(low, high)) for _, (low, high) in zip(range(len(self.ranges)), self.ranges)]
        return self.constructor(*randoms)(image)
    

class PretrainingDatasetWrapper(Dataset):
    def __init__(self, ds: Dataset, l: labels, target_size=(512,512), debug=False):
        super().__init__()
        self.ds = ds
        self.labels = labels
        self.debug = debug
        self.target_size = target_size
        if debug:
            print("DATASET IN DEBUG MODE")
        
        # I will be using network pre-trained on ImageNet first, which uses this normalization.
        # Remove this, if you're training from scratch or apply different transformations accordingly
        self.preprocess = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        
        random_resized_rotation = WrapWithRandomParams(lambda angle: ResizedRotation(angle, target_size), [(0.0, 360.0)])
        self.randomize = transforms.Compose([
            transforms.RandomResizedCrop(target_size, scale=(330/512, 330/512), ratio=(1.0, 1.0)),
            transforms.RandomChoice([
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.Lambda(random_rotate)
            ]),
            transforms.RandomApply([
                random_resized_rotation
            ], p=0.6),
            transforms.RandomApply([
                transforms.ColorJitter(brightness=0.5, contrast=0.6, saturation=0.5, hue=0.2)
            ], p=0.8)
        ])
    
    def __len__(self): return len(self.ds)
    
    def __getitem_internal__(self, idx, preprocess=True):
        this_image_raw_path = self.ds[idx]
        label = self.labels[idx]
        this_image_raw = Image.open(this_image_raw_path)
        if self.debug:
            random.seed(idx)
            t1 = self.randomize(this_image_raw)
            random.seed(idx + 1)
            t2 = self.randomize(this_image_raw)
        else:
            t1 = self.randomize(this_image_raw)
            t2 = self.randomize(this_image_raw)
        
        if preprocess:
            t1 = self.preprocess(t1)
            t2 = self.preprocess(t2)
        else:
            t1 = transforms.ToTensor()(t1)
            t2 = transforms.ToTensor()(t2)
            
        return (t1, t2), torch.tensor(label)

    def __getitem__(self, idx):
        return self.__getitem_internal__(idx, True)
    
    def raw(self, idx):
        return self.__getitem_internal__(idx, False)
    
    
    
from efficientnet_pytorch import EfficientNet
class ImageEmbedding(nn.Module):       
    class Identity(nn.Module):
        def __init__(self): super().__init__()

        def forward(self, x):
            return x
    
        
    def __init__(self, embedding_size=1024):
        super().__init__()
        
        #base_model = EfficientNet.from_pretrained("efficientnet-b2")
        base_model = EfficientNet.from_name("efficientnet-b2")
        internal_embedding_size = base_model._fc.in_features
        base_model._fc = ImageEmbedding.Identity()
        
        self.embedding = base_model
        
        self.projection = nn.Sequential(
            nn.Linear(in_features=internal_embedding_size, out_features=embedding_size),
            nn.ReLU(),
            nn.Linear(in_features=embedding_size, out_features=embedding_size)
        )

    def calculate_embedding(self, image):
        return self.embedding(image)

    def forward(self, X):
        image = X
        embedding = self.calculate_embedding(image)
        projection = self.projection(embedding)
        return embedding, projection

hparams = Namespace(
    lr=0.0001,
    epochs=1,
    batch_size=64,
    train_size=37184,
    validation_size=4150
)
    
class ImageEmbeddingModule(pl.LightningModule):
    def __init__(self, hparams = hparams):
        hparams = Namespace(**hparams) if isinstance(hparams, dict) else hparams
        super().__init__()
        #self.hparams = hparams
        self.model = ImageEmbedding()
        self.loss = SupConLoss()
    
    def total_steps(self):
        return len(self.train_dataloader()) // hparams.epochs
    
    def train_dataloader(self):
        return DataLoader(PretrainingDatasetWrapper(data,labels,
                                             debug=getattr(hparams, "debug", False)),
                          batch_size=hparams.batch_size, 
                          num_workers=4,#cpu_count(),
                          sampler=SubsetRandomSampler(list(range(hparams.train_size))),
                         drop_last=True)
    
    def val_dataloader(self):
        return DataLoader(PretrainingDatasetWrapper(data,labels,
                                            debug=getattr(hparams, "debug", False)),
                          batch_size=hparams.batch_size, 
                          shuffle=False,
                          num_workers=4,#cpu_count(),
                          sampler=SequentialSampler(list(range(hparams.train_size + 1, hparams.train_size + hparams.validation_size))),
                         drop_last=True)
    
    def forward(self, X):
        return self.model(X)
    
    def step(self, batch, step_name = "train"):
        (X, Y), labels = batch
        embX, projectionX = self.forward(X)
        embY, projectionY = self.forward(Y)
        z_i = F.normalize(projectionX , dim=1)
        z_j = F.normalize(projectionY, dim=1)
        projX = torch.reshape(z_i,(z_i.shape[0],1,z_i.shape[1]))
        projY = torch.reshape(z_j,(z_j.shape[0],1,z_j.shape[1]))
        features = torch.cat([projX, projY], dim=1)
        loss = self.loss(features=features,labels=labels)
        loss_key = f"{step_name}_loss"
        tensorboard_logs = {loss_key: loss}
        self.log("loss" if step_name == "train" else loss_key, loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        #return { ("loss" if step_name == "train" else loss_key): loss, 'log': tensorboard_logs,
                        #"progress_bar": {loss_key: loss}}
        return loss
    
    def training_step(self, batch, batch_idx):
        return self.step(batch, "train")
    
    def validation_step(self, batch, batch_idx):
        return self.step(batch, "val")
    
    def validation_end(self, outputs):
        if len(outputs) == 0:
            return {"val_loss": torch.tensor(0)}
        else:
            loss = torch.stack([x["val_loss"] for x in outputs]).mean()
            return {"val_loss": loss, "log": {"val_loss": loss}}

    def configure_optimizers(self):
        optimizer = RMSprop(self.model.parameters(), lr=hparams.lr)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=15, verbose=True)
        return {
           'optimizer': optimizer,
           'lr_scheduler': scheduler,
           'monitor': 'val_loss'
       }
hparams = Namespace(
    lr=0.0001,
    epochs=500,
    batch_size=22,
    train_size=33396,
    validation_size=3708
)

Length of data : 14226
Length of labels : 14226


In [3]:
model = ImageEmbeddingModule.load_from_checkpoint(
    checkpoint_path="/workstation/home/bijoy/data_from_b170007ec/Programs/Bhanu/SCLEARNING/Checkpoints/model-v3.ckpt"
)

In [4]:
from io import BytesIO
from matplotlib import cm
from skimage import img_as_ubyte
from PIL import Image
import base64
import sklearn
import llvmlite
import umap.umap_ as umap
from matplotlib.colors import ListedColormap


def draw_umap(n_neighbors=15, min_dist=0.1, n_components=2, metric='euclidean', title=''):
    fit = umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        n_components=n_components,
        metric=metric
    )
    classes = [0,1,2,3,4,5]
    colors=ListedColormap(["green", "orange",  "blue", "#550011", "purple", "red"])
    u = fit.fit_transform(latent_space);
    fig = plt.figure(figsize=(15,15))
    try:
        if n_components == 1:
            ax = fig.add_subplot(111)
            scatter = ax.scatter(u[:,0], range(len(u)), c = labels,cmap=colors)
        if n_components == 2:
            ax = fig.add_subplot(111)
            scatter = ax.scatter(u[:,0], u[:,1], s=1, c = labels,cmap=colors)
        if n_components == 3:
            ax = fig.add_subplot(111, projection='3d')
            scatter = ax.scatter(u[:,0], u[:,1], u[:,2], s=1, c = labels,cmap=colors)
        plt.legend(handles=scatter.legend_elements()[0], labels=classes)
        plt.title(title, fontsize=18)
        plt.savefig("Umap_plot_SClearning_test_{}".format(n_neighbors))
        plt.show()
    except:
        pass
    return u

In [11]:
preprocess = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
latent_space = []

from tqdm import tqdm
for path in tqdm(data[:2]):
    projection = np.zeros((1, 1024))
    this_image_raw = Image.open(path)
    img = tvf.resize(this_image_raw, (512,512))
    t1 = preprocess(img)
    t2 = t1.unsqueeze(0)
    embeddings, projections = model(t2)
    projection[0,:] = projections.cpu().detach().numpy()
    latent_space.append(projection)
latent_space = np.array(latent_space)
print(latent_space.shape)

100%|██████████| 2/2 [00:08<00:00,  4.00s/it]

(2, 1, 1024)


In [12]:
import pickle
image_features_arr = np.rollaxis(latent_space,1,0)
print(image_features_arr.shape)
image_features_arr = image_features_arr[0,:,:]

np.savetxt('feature_vectors_samples.txt',image_features_arr)
#feature_vectors = np.loadtxt('feature_vectors.txt')
pickle.dump(image_features_arr, open('feature_vectors_samples.pkl', 'wb'))

(1, 2, 1024)


In [ ]:
u = draw_umap(n_neighbors=10000, min_dist = 0.1, n_components=2, metric='euclidean', title='UMAP-TEST DATASET')

In [4]:
df.head()

,Path,Noise,Data_type,Predicted_label
0,/workstation/home/bijoy/data_from_b170007ec/Da...,0,Train,0
1,/workstation/home/bijoy/data_from_b170007ec/Da...,0,Train,0
2,/workstation/home/bijoy/data_from_b170007ec/Da...,0,Train,0
3,/workstation/home/bijoy/data_from_b170007ec/Da...,0,Train,0
4,/workstation/home/bijoy/data_from_b170007ec/Da...,0,Train,0


In [10]:
import pandas as pd
df = pd.read_csv("/workstation/home/bijoy/data_from_b170007ec/Programs/Bhanu/SCLEARNING/Output_csv/Model_binary_v3.csv", usecols=range(1,5))
predicted_noise = df[df["Predicted_label"]==1]
cervix = df[(df["Predicted_label"] == 0)]
cervix = cervix[cervix["Data_type"] == "Train"]